# Decentralized SGD Algorithm

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import random
import numpy as np

## For reproducibility

In [ ]:
torch.manual_seed(0)
np.random.seed(0)
random.seed(0)

## Define a simple neural network

In [ ]:
class SimpleNN(nn.Module):
    def __init__(self, input_size=20, hidden_size=50, output_size=2):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, output_size)
    
    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out

# Generate synthetic dataset
def generate_synthetic_data(num_samples=1000, input_size=20, num_classes=2):
    X = torch.randn(num_samples, input_size)
    Y = torch.randint(0, num_classes, (num_samples,))
    return X, Y

## # Define the Peer class

In [ ]:
class Peer:
    def __init__(self, peer_id, model, train_loader, lr=0.01):
        self.peer_id = peer_id
        self.model = model
        self.train_loader = train_loader
        self.optimizer = optim.SGD(self.model.parameters(), lr=lr)
        self.criterion = nn.CrossEntropyLoss()
        self.neighbors = []  # To be set based on network topology

    def set_neighbors(self, neighbors):
        self.neighbors = neighbors

    def local_train(self, epochs=1):
        self.model.train()
        for epoch in range(epochs):
            for batch_idx, (data_batch, target) in enumerate(self.train_loader):
                data_batch, target = data_batch.float(), target.long()
                self.optimizer.zero_grad()
                outputs = self.model(data_batch)
                loss = self.criterion(outputs, target)
                loss.backward()
                self.optimizer.step()

    def average_with_neighbors(self):
        # Collect parameters from neighbors
        neighbor_params = []
        for neighbor in self.neighbors:
            neighbor_params.append([param.data.clone() for param in neighbor.model.parameters()])

        # Include own parameters
        own_params = [param.data.clone() for param in self.model.parameters()]
        neighbor_params.append(own_params)


   ## Compute average

In [ ]:

        avg_params = []
        num_sources = len(neighbor_params)
        for param_idx in range(len(own_params)):
            avg = torch.zeros_like(own_params[param_idx])
            for source in neighbor_params:
                avg += source[param_idx]
            avg /= num_sources
            avg_params.append(avg)

        # Update own parameters
        for param, avg in zip(self.model.parameters(), avg_params):
            param.data.copy_(avg)

    def get_parameters(self):
        return [param.data.clone() for param in self.model.parameters()]


## Define network topology (e.g., Ring)

In [ ]:
def create_ring_topology(peers):
    num_peers = len(peers)
    for i, peer in enumerate(peers):
        left_neighbor = peers[(i - 1) % num_peers]
        right_neighbor = peers[(i + 1) % num_peers]
        peer.set_neighbors([left_neighbor, right_neighbor])

# Alternatively, create a fully connected topology
def create_fully_connected_topology(peers):
    for peer in peers:
        peer.set_neighbors([p for p in peers if p != peer])

# Create peers with local datasets
def initialize_peers(num_peers=5, input_size=20, hidden_size=50, output_size=2, 
                    num_samples_per_peer=1000, batch_size=32, lr=0.01):
    peers = []
    for i in range(num_peers):
        model = SimpleNN(input_size, hidden_size, output_size)
        X, Y = generate_synthetic_data(num_samples=num_samples_per_peer, input_size=input_size, num_classes=output_size)
        dataset = data.TensorDataset(X, Y)
        train_loader = data.DataLoader(dataset, batch_size=batch_size, shuffle=True)
        peer = Peer(peer_id=i, model=model, train_loader=train_loader, lr=lr)
        peers.append(peer)
    return peers

## Evaluate the model on a test set

In [ ]:

def evaluate(peers, test_loader):
    # Aggregate parameters (e.g., average all peer models)
    avg_model = SimpleNN()
    num_peers = len(peers)
    for param in avg_model.parameters():
        param.data = torch.zeros_like(param)

    for peer in peers:
        for avg_param, param in zip(avg_model.parameters(), peer.model.parameters()):
            avg_param.data += param.data

    for param in avg_model.parameters():
        param.data /= num_peers

    # Evaluate the averaged model
    avg_model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for data_batch, target in test_loader:
            data_batch, target = data_batch.float(), target.long()
            outputs = avg_model(data_batch)
            _, predicted = torch.max(outputs.data, 1)
            total += target.size(0)
            correct += (predicted == target).sum().item()

    accuracy = 100 * correct / total
    print(f'Aggregated Model Accuracy on Test Set: {accuracy:.2f}%')


## Main training loop

In [ ]:

def decentralized_sgd(num_peers=5, input_size=20, hidden_size=50, output_size=2, 
                     num_samples_per_peer=1000, batch_size=32, lr=0.01, 
                     num_rounds=50, local_epochs=1, topology='ring'):
    # Initialize peers
    peers = initialize_peers(num_peers, input_size, hidden_size, output_size, 
                             num_samples_per_peer, batch_size, lr)

    # Define network topology
    if topology == 'ring':
        create_ring_topology(peers)
    elif topology == 'fully_connected':
        create_fully_connected_topology(peers)
    else:
        raise ValueError("Unsupported topology. Choose 'ring' or 'fully_connected'.")

    # Create a global test set
    X_test, Y_test = generate_synthetic_data(num_samples=1000, input_size=input_size, num_classes=output_size)
    test_dataset = data.TensorDataset(X_test, Y_test)
    test_loader = data.DataLoader(test_dataset, batch_size=64, shuffle=False)

    # Training rounds
    for round_num in range(1, num_rounds + 1):
        print(f'Round {round_num}/{num_rounds}')
        
        # Local training
        for peer in peers:
            peer.local_train(epochs=local_epochs)
        
        # Model averaging with neighbors
        for peer in peers:
            peer.average_with_neighbors()

        # Optionally, evaluate periodically
        if round_num % 10 == 0 or round_num == 1:
            evaluate(peers, test_loader)

    # Final evaluation
    print("Final Evaluation:")
    evaluate(peers, test_loader)

if __name__ == "__main__":
    decentralized_sgd(
        num_peers=5,
        input_size=20,
        hidden_size=50,
        output_size=2,
        num_samples_per_peer=1000,
        batch_size=32,
        lr=0.01,
        num_rounds=50,
        local_epochs=1,
        topology='ring'  
    )
